# 2012-2015년 데이터로 모델 만들기(남자)

In [135]:
import pandas as pd
from itertools import *

In [62]:
dataset_male.columns

Index(['area', 'tma', 'yyyy', 'mm', 'dd', 'weekday', 'sum_gsr', 'sum_rn',
       'avg_ps', 'max_ps', 'min_ps', 'so2', 'o3', 'pm10', 'sex', 'elderly_pop',
       'total_pop', 'elderly_ratio', 'avg_age', 'min_max_ps', 'D/R',
       'heat_wave', 'cold_wave', 'min_hum', 'max_hum', 'avg_hum', 'diff_hum',
       'min_temp', 'max_temp', 'avg_temp', 'diff_temp', 'pm10_3b', 'pm10_7b',
       'pm10_31b', 'frequency'],
      dtype='object')

In [220]:
dataset_m = pd.read_csv('Z:/공모전/날씨빅데이터콘테스트(공유)/data/test/hospital_male_0802_test2.csv', encoding='cp949')
dataset_male = dataset_m[dataset_m['yyyy'].astype(str).str.contains('2012|2013|2014|2015')].reset_index(drop=True)#2016년 전만 추출

m_cols = ['area','mm','weekday','heat_wave','cold_wave','elderly_ratio','sex', 
        'sum_gsr', 'total_pop',
        'pm10_7b','diff_hum','diff_temp','avg_ps', 'o3','min_max_ps', 'avg_hum','frequency']  # 17

m_cols_1 = ['area', 'mm', 'weekday', 'heat_wave', 'cold_wave', 'total_pop', 'elderly_ratio', 
            'pm10_7b', 'diff_hum', 'diff_temp', 'avg_ps', 
            'o3', 'avg_age', 'min_max_ps', 'avg_hum', 'frequency']  # 16

male_df = dataset_male[m_cols]
male_df

,area,mm,weekday,heat_wave,cold_wave,elderly_ratio,sex,sum_gsr,sum_rn,total_pop,pm10_7b,diff_hum,diff_temp,avg_ps,o3,min_max_ps,avg_hum,frequency
0,강원,1,6,0,0,12.460044,1,0.000000,0.173611,772718,30.158333,65.0,12.3,1023.886667,0.027750,6.340000,58.333333,3.0
1,경기,1,6,0,0,7.432174,1,0.000000,0.037736,6020446,28.537404,27.0,7.9,1024.875000,0.023165,5.500000,50.733333,4.0
2,경남,1,6,0,0,9.179443,1,0.000000,0.065789,1665308,26.719907,47.0,9.9,1022.950000,0.027277,5.200000,44.916667,2.0
3,경북,1,6,0,0,12.336406,1,0.000702,0.721053,1354649,25.336806,70.0,12.0,1023.268000,0.027274,5.688000,52.377778,6.0
4,광주,1,6,0,0,7.679253,1,0.000000,0.700000,726529,18.958333,15.0,3.9,1025.250000,0.027300,4.100000,75.833333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24832,전남,12,3,0,0,16.198981,1,0.000000,1.608108,953881,46.708333,65.0,12.3,1031.053846,0.035985,6.892308,66.764033,0.0
24833,전북,12,3,0,0,14.539347,1,1.305806,0.967742,930255,72.548882,50.0,10.0,1031.300000,0.028738,7.014286,73.750000,1.0
24834,제주,12,3,0,0,11.063466,1,0.000000,0.971429,313428,56.156250,33.0,4.9,1030.792308,0.044719,7.523077,59.915541,0.0
24835,충남,12,3,0,0,13.477024,1,1.757273,0.196970,1054439,79.543252,55.0,9.0,1031.513333,0.024484,6.800000,74.725225,0.0


In [64]:
# male_data = male_df.sample(frac=0.9, random_state=786) 
# male_data_unseen = male_df.drop(male_data.index) 

# male_data.reset_index(drop=True, inplace=True) 
# male_data_unseen.reset_index(drop=True, inplace=True) 

# print('Data for Modeling: ' + str(male_data.shape)) 
# print('Unseen Data For Predictions: ' + str(male_data_unseen.shape)) 


Data for Modeling: (22353, 31)
Unseen Data For Predictions: (2484, 31)


## 모델 환경 설정 (Settinf up Environment in PyCaret)

- pycaret을 사용하기 전에 pycaret에 맞게 데이터를 설정
- set_up() 함수를 사용
- 기본적으로 data와 target을 입력

In [221]:
# 자동으로 데이터 유형 지정 
from pycaret.regression import *
# s = setup(male_data, target = 'frequency', session_id=123, transformation = True)

s1 = setup(data = male_df, train_size= 0.9, target = 'frequency', session_id=123, 
           data_split_stratify = True, fold_strategy = 'timeseries', fold_shuffle = True,
           normalize = True, verbose = True)

,Description,Value
0,session_id,123
1,Target,frequency
2,Original Data,"(24837, 18)"
3,Missing Values,False
4,Numeric Features,11
5,Categorical Features,6
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(22353, 54)"


#### set_up(): pycaret을 사용하기 위한 data setting

- session_id: random_state와 같은 개념으로 같은 결과가 나올 수 있게 seed를 고정
- data: train 데이터 입력
- target = target 변수 이름을 입력

## 모델 생성 - 비교

- 여러 모델을 적합하여 성능을 비교하는 단계
- compare_models(): 다양한 모델 적합 후 성능 비교
    - fold: cross_validation의 fold를 지정 (default = 10)
    - sort: 정렬기준 지표 설정
    - n_select: 상위 n개의 모델 결과만 출력

In [222]:
# et_male = create_model('et')
male_best_model_5 = compare_models(sort='RMSE', n_select = 5)
male_tuned_top5 = [tune_model(i, optimize = 'RMSE', n_iter=15) for i in male_best_model_5]
model_male = blend_models(estimator_list=male_tuned_top5, optimize = 'RMSE')

# final_model = finalize_model(model_male)
# prediction = predict_model(final_model, data=male_data_unseen)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.9524,1.6710,1.2927,0.4088,0.5068,0.4375
1,0.8968,1.4707,1.2127,0.4662,0.4822,0.4221
2,0.9258,1.5587,1.2485,0.4670,0.4917,0.4404
3,0.9263,1.6575,1.2875,0.4535,0.4934,0.4184
4,0.9085,1.4547,1.2061,0.4470,0.4973,0.4395
5,0.9069,1.5359,1.2393,0.4526,0.4841,0.4274
6,0.9119,1.5653,1.2511,0.4374,0.4888,0.4486
7,0.9319,1.6222,1.2736,0.4580,0.4883,0.4208
8,0.9018,1.5712,1.2535,0.5027,0.4867,0.4103


In [223]:
predict_model(model_male)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,0.9095,1.5346,1.2388,0.4665,0.4858,0.4320


,elderly_ratio,sum_gsr,sum_rn,total_pop,pm10_7b,diff_hum,diff_temp,avg_ps,o3,min_max_ps,...,weekday_6,heat_wave_0,heat_wave_1,heat_wave_2,cold_wave_0,cold_wave_1,cold_wave_2,sex_1,frequency,Label
0,0.517821,-0.247619,0.482381,-0.925113,-1.591711,-0.806567,-0.238021,0.168007,-0.016351,-0.349413,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.187106
1,-0.204838,-0.247619,-0.264769,-0.913152,0.043950,-0.933527,-1.346980,-3.162168,-1.495632,5.551817,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.142425
2,-0.047736,-0.247619,3.670598,-0.772153,0.472167,-1.250925,-1.256453,-4.712589,0.387089,5.512272,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.523018
3,1.240884,-0.246751,-0.032665,-0.098313,0.276832,-0.933527,-0.396444,0.145093,-0.063742,-0.343195,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,5.0,1.975483
4,-0.516907,-0.247619,-0.297254,0.102625,-0.392441,1.224785,1.391469,0.280179,-0.319036,-0.317134,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,4.0,1.918842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,-0.057750,-0.247619,-0.297254,2.224295,-0.507478,-0.044810,-0.215390,0.273438,-0.173169,2.211469,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,4.0,3.588622
2480,-1.403580,-0.247619,1.577582,-0.577058,-0.160807,-1.949202,-1.233821,0.278755,-0.773420,-0.357007,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.304138
2481,2.284241,-0.247619,-0.297254,-0.354331,0.504385,1.224785,-0.260653,-0.120629,-0.297493,-0.286800,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,1.226320
2482,0.309334,-0.247619,-0.297254,-0.457164,0.192011,0.463028,1.572524,0.452895,-2.121288,3.992178,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,1.278364


- 1.2384 / 0.4668 (avg_age x)
- 1.2383 / 0.4669 (avg_age, max_hum x)
- 1.2384 / 0.4669 (max_hum x)
- 1.2384 / 0.4668 (sum_gsr x)
- 1.2384 / 0.4668 (normalize x)
- 1.2358 / 0.4691 (total_pop x)
- 1.2388 / 0.4665 (sum_rn o)

# 2016년 예측(남자)

In [224]:
testset_male = dataset_m[dataset_m['yyyy'].astype(str).str.contains('2016')].reset_index(drop=True)#2016년만 추출

test_male2 = testset_male[m_cols]
test_male2

,area,mm,weekday,heat_wave,cold_wave,elderly_ratio,sex,sum_gsr,sum_rn,total_pop,pm10_7b,diff_hum,diff_temp,avg_ps,o3,min_max_ps,avg_hum,frequency
0,강원,1,4,0,0,14.022671,1,0.000000,0.000000,781185,61.954167,73.0,18.9,1030.230000,0.024237,8.010000,58.196911,NaN
1,경기,1,4,0,0,8.966558,1,0.000000,0.000000,6306634,42.487621,45.0,12.1,1031.633333,0.014370,7.016667,70.844595,NaN
2,경남,1,4,0,0,10.962802,1,1.634500,0.000000,1695187,29.128224,79.0,17.7,1031.175000,0.029582,5.616667,52.099421,NaN
3,경북,1,4,0,0,14.384461,1,0.001404,0.000000,1356568,38.389385,66.0,20.0,1030.309524,0.033232,6.233333,57.314554,NaN
4,광주,1,4,0,0,9.572120,1,0.000000,0.000000,729995,20.475000,40.0,10.0,1032.200000,0.025175,5.850000,62.837838,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6217,전남,12,5,0,0,16.677050,1,0.000000,0.000000,951889,33.635000,49.0,13.7,1030.607692,0.025507,2.573333,65.555094,NaN
6218,전북,12,5,0,0,15.011563,1,0.820968,0.000000,927505,52.149758,42.0,14.5,1030.586667,0.021604,3.466667,72.161486,NaN
6219,제주,12,5,0,0,11.278426,1,0.000000,0.000000,322545,28.460474,31.0,8.0,1029.900000,0.039094,3.046667,60.569257,NaN
6220,충남,12,5,0,0,13.768860,1,1.973636,0.045455,1064765,71.473958,41.0,11.7,1030.100000,0.019109,2.520000,75.118243,NaN


In [225]:
test_male3 = test_male2.drop(columns=['frequency'])

In [226]:
male_final_model = finalize_model(model_male)
prediction_male = predict_model(male_final_model, data = test_male3)

In [10]:
# prediction_male['Label'] = prediction_male['Label'].round(0)
# prediction_male[['Label']]

In [ ]:
evaluate_model(male_final_model)

# 2012-2015년 데이터로 모델 만들기(여자)

In [232]:
dataset_f = pd.read_csv('Z:/공모전/날씨빅데이터콘테스트(공유)/data/test/hospital_female_0802_test2.csv', encoding='cp949')
dataset_female = dataset_f[dataset_f['yyyy'].astype(str).str.contains('2012|2013|2014|2015')].reset_index(drop=True)#2016년 전만 추출
# female_df = dataset_female[['area','mm','weekday','heat_wave','cold_wave','total_pop','elderly_ratio',
#           'pm10_7b','diff_hum','diff_temp','avg_ps',
#           'avg_age','min_max_ps', 'sum_rn','frequency']]

f_cols = ['area','mm','weekday','heat_wave','cold_wave','total_pop','elderly_ratio','sex', 
        'sum_gsr','max_hum', 
        'pm10_7b','diff_hum','diff_temp','avg_ps',
        'o3','avg_age','min_max_ps', 'avg_hum','frequency']

# cols = ['area', 'mm', 'weekday', 'sum_gsr', 'sum_rn',
#        'avg_ps', 'max_ps', 'min_ps', 'so2', 'o3', 'pm10', 'sex', 'elderly_pop',
#        'total_pop', 'elderly_ratio', 'avg_age', 'min_max_ps',
#        'heat_wave', 'cold_wave', 'min_hum', 'max_hum', 'avg_hum', 'diff_hum',
#        'min_temp', 'max_temp', 'avg_temp', 'diff_temp', 'pm10_3b', 'pm10_7b',
#        'pm10_31b', 'frequency']

female_df = dataset_female[f_cols]
female_df

,area,mm,weekday,heat_wave,cold_wave,total_pop,elderly_ratio,sex,sum_gsr,max_hum,pm10_7b,diff_hum,diff_temp,avg_ps,o3,avg_age,min_max_ps,avg_hum,frequency
0,강원,1,6,0,0,762617,18.053361,2,0.000000,95.0,30.158333,65.0,12.3,1023.886667,0.027750,42.4,6.340000,58.333333,3.0
1,경기,1,6,0,0,5928150,10.649039,2,0.000000,65.0,28.537404,27.0,7.9,1024.875000,0.023165,37.7,5.500000,50.733333,5.0
2,경남,1,6,0,0,1644251,15.076105,2,0.000000,78.0,26.719907,47.0,9.9,1022.950000,0.027277,40.5,5.200000,44.916667,1.0
3,경북,1,6,0,0,1342455,19.248690,2,0.000702,90.0,25.336806,70.0,12.0,1023.268000,0.027274,43.2,5.688000,52.377778,1.0
4,광주,1,6,0,0,738784,11.037868,2,0.000000,85.0,18.958333,15.0,3.9,1025.250000,0.027300,37.6,4.100000,75.833333,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24832,전남,12,3,0,0,955115,24.847060,2,0.000000,95.0,46.708333,65.0,12.3,1031.053846,0.035985,45.9,6.892308,66.764033,3.0
24833,전북,12,3,0,0,939456,21.104873,2,1.305806,95.0,72.548882,50.0,10.0,1031.300000,0.028738,44.2,7.014286,73.750000,0.0
24834,제주,12,3,0,0,310967,16.470236,2,0.000000,78.0,56.156250,33.0,4.9,1030.792308,0.044719,41.5,7.523077,59.915541,0.0
24835,충남,12,3,0,0,1023210,19.459055,2,1.757273,95.0,79.543252,55.0,9.0,1031.513333,0.024484,43.1,6.800000,74.725225,0.0


In [39]:
# female_data = female_df.sample(frac=0.9, random_state=786) 
# female_data_unseen = female_df.drop(female_data.index) 

# female_data.reset_index(drop=True, inplace=True) 
# female_data_unseen.reset_index(drop=True, inplace=True) 

# print('Data for Modeling: ' + str(female_data.shape)) 
# print('Unseen Data For Predictions: ' + str(female_data_unseen.shape)) 



Data for Modeling: (22353, 15)
Unseen Data For Predictions: (2484, 15)


In [233]:
# 자동으로 데이터 유형 지정 
from pycaret.regression import *
# s = setup(female_data, target = 'frequency', session_id=123, transformation = True)

s2 = setup(data = female_df, train_size= 0.9, target = 'frequency', session_id=123, 
          data_split_stratify = True, fold_strategy = 'timeseries', fold_shuffle = True,
          normalize = True, verbose = True)

,Description,Value
0,session_id,123
1,Target,frequency
2,Original Data,"(24837, 19)"
3,Missing Values,False
4,Numeric Features,12
5,Categorical Features,6
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(22353, 55)"


In [238]:
# et_female = create_model('et')
female_best_model_5 = compare_models(sort='RMSE', n_select = 5)
# female_tuned_top5 = [tune_model(i, optimize = 'RMSE', n_iter=10) for i in female_best_model_5]
# model_female = stack_models(tuned_top5)
model_female = blend_models(female_best_model_5, optimize = 'RMSE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.9227,1.5904,1.2611,0.4185,0.4902,0.4581
1,0.9086,1.5858,1.2593,0.4271,0.4861,0.4564
2,0.8967,1.5165,1.2314,0.4492,0.4800,0.4504
3,0.9378,1.5986,1.2643,0.4212,0.5005,0.4592
4,0.9019,1.5076,1.2279,0.4277,0.4946,0.4594
5,0.8993,1.4863,1.2191,0.4343,0.4819,0.4463
6,0.8805,1.4173,1.1905,0.4464,0.4831,0.4360
7,0.9525,1.6749,1.2942,0.4472,0.4906,0.4623
8,0.9255,1.6077,1.2679,0.4811,0.4876,0.4551


In [239]:
predict_model(model_female)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,0.9037,1.4638,1.2099,0.4578,0.4852,0.4605


,total_pop,elderly_ratio,sum_gsr,max_hum,pm10_7b,diff_hum,diff_temp,avg_ps,o3,avg_age,...,weekday_6,heat_wave_0,heat_wave_1,heat_wave_2,cold_wave_0,cold_wave_1,cold_wave_2,sex_2,frequency,Label
0,2.296820,-0.470720,-0.249035,-0.853590,-1.352574,-0.362260,-0.440325,0.150471,0.368933,0.100486,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,3.611964
1,2.327517,-0.890206,-0.249035,-1.833039,-0.271332,-1.315606,-0.892057,0.356860,-1.673333,-0.515953,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,3.086782
2,-0.606163,-1.722123,-0.249035,0.125860,-0.423088,-2.014726,-1.569656,0.244463,-0.632347,-1.653991,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.488876
3,-0.600223,-1.525192,-0.249035,-0.363865,-0.023523,-0.997824,-0.259632,0.164274,1.468776,-1.227227,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.531731
4,-0.461065,0.364575,-0.249035,0.615584,1.695112,0.273304,0.259860,-4.708443,2.088184,0.384995,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.014958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,-0.313613,0.817277,0.334092,0.615584,-0.306386,0.908869,0.372793,0.279168,-0.339689,0.574669,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.915237
2480,-0.922259,0.642916,-0.249035,0.419694,-0.925822,0.781756,0.914872,0.315286,-0.085885,0.195322,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.144499
2481,2.868496,-1.166740,-0.249035,0.125860,0.563317,-0.616485,0.463139,0.298361,-1.534632,-1.464319,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,6.0,4.439882
2482,2.335220,-0.944682,1.686391,-0.363865,-0.681227,-1.633388,-0.892057,0.245778,-1.004744,-0.610788,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,3.548570


- 1.2099	0.4578
- 1.2115	0.4563	(tuning)
- 1.2099	0.4578 optimize setting

# 2016년 예측(여자)

In [240]:
testset_female = dataset_f[dataset_f['yyyy'].astype(str).str.contains('2016')].reset_index(drop=True)#2016년만 추출

test_female2 = testset_female[f_cols]
test_female2

,area,mm,weekday,heat_wave,cold_wave,total_pop,elderly_ratio,sex,sum_gsr,max_hum,pm10_7b,diff_hum,diff_temp,avg_ps,o3,avg_age,min_max_ps,avg_hum,frequency
0,강원,1,4,0,0,768008,19.916980,2,0.000000,95.0,61.954167,73.0,18.9,1030.230000,0.024237,44.5,8.010000,58.196911,NaN
1,경기,1,4,0,0,6229840,12.194759,2,0.000000,90.0,42.487621,45.0,12.1,1031.633333,0.014370,39.7,7.016667,70.844595,NaN
2,경남,1,4,0,0,1669577,16.753405,2,1.634500,94.0,29.128224,79.0,17.7,1031.175000,0.029582,42.4,5.616667,52.099421,NaN
3,경북,1,4,0,0,1344592,21.253882,2,0.001404,90.0,38.389385,66.0,20.0,1030.309524,0.033232,45.1,6.233333,57.314554,NaN
4,광주,1,4,0,0,742807,13.091826,2,0.000000,80.0,20.475000,40.0,10.0,1032.200000,0.025175,39.7,5.850000,62.837838,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6217,전남,12,5,0,0,952025,25.227174,2,0.000000,89.0,33.635000,49.0,13.7,1030.607692,0.025507,46.4,2.573333,65.555094,NaN
6218,전북,12,5,0,0,937286,21.548385,2,0.820968,87.0,52.149758,42.0,14.5,1030.586667,0.021604,44.7,3.466667,72.161486,NaN
6219,제주,12,5,0,0,319052,16.552474,2,0.000000,76.0,28.460474,31.0,8.0,1029.900000,0.039094,41.9,3.046667,60.569257,NaN
6220,충남,12,5,0,0,1031962,19.719912,2,1.973636,95.0,71.473958,41.0,11.7,1030.100000,0.019109,43.5,2.520000,75.118243,NaN


In [241]:
test_female3 = test_female2.drop(columns=['frequency'])

In [242]:
female_final_model = finalize_model(model_female)
prediction_female = predict_model(female_final_model, data = test_female3)

In [19]:
# prediction_female['Label'] = prediction_female['Label'].round(0)
# prediction_female[['Label']]

# 남자 여자 예측 데이터 합치기

In [243]:
hap_male = pd.concat([testset_male[['area','tma','sex']],prediction_male['Label']], axis=1)
hap_male

,area,tma,sex,Label
0,강원,2016-01-01,1,1.164587
1,경기,2016-01-01,1,4.519052
2,경남,2016-01-01,1,1.982569
3,경북,2016-01-01,1,1.877962
4,광주,2016-01-01,1,0.755054
...,...,...,...,...
6217,전남,2016-12-31,1,1.231277
6218,전북,2016-12-31,1,1.035988
6219,제주,2016-12-31,1,0.212228
6220,충남,2016-12-31,1,0.983651


In [244]:
hap_female = pd.concat([testset_female[['area','tma','sex']],prediction_female['Label']], axis=1)
hap_female

,area,tma,sex,Label
0,강원,2016-01-01,2,1.084037
1,경기,2016-01-01,2,4.276632
2,경남,2016-01-01,2,2.104159
3,경북,2016-01-01,2,1.919381
4,광주,2016-01-01,2,0.771784
...,...,...,...,...
6217,전남,2016-12-31,2,1.496123
6218,전북,2016-12-31,2,1.221665
6219,제주,2016-12-31,2,0.224163
6220,충남,2016-12-31,2,1.014732


In [245]:
verify = pd.read_csv('Z:/공모전/날씨빅데이터콘테스트(공유)/hospital_data/2-2_검증데이터셋.csv', encoding='cp949')
verify

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,NaN
1,2016-01-01,경기,1,NaN
2,2016-01-01,경남,1,NaN
3,2016-01-01,경북,1,NaN
4,2016-01-01,광주,1,NaN
...,...,...,...,...
12439,2016-12-31,전남,2,NaN
12440,2016-12-31,전북,2,NaN
12441,2016-12-31,제주,2,NaN
12442,2016-12-31,충남,2,NaN


In [246]:
hap_male.sort_values(['tma', 'area']).reset_index(drop=True)
hap_female.sort_values(['tma', 'area']).reset_index(drop=True)

,area,tma,sex,Label
0,강원,2016-01-01,2,1.084037
1,경기,2016-01-01,2,4.276632
2,경남,2016-01-01,2,2.104159
3,경북,2016-01-01,2,1.919381
4,광주,2016-01-01,2,0.771784
...,...,...,...,...
6217,전남,2016-12-31,2,1.496123
6218,전북,2016-12-31,2,1.221665
6219,제주,2016-12-31,2,0.224163
6220,충남,2016-12-31,2,1.014732


In [247]:
hap = pd.concat([hap_male, hap_female], axis=0).reset_index(drop=True)
hap

,area,tma,sex,Label
0,강원,2016-01-01,1,1.164587
1,경기,2016-01-01,1,4.519052
2,경남,2016-01-01,1,1.982569
3,경북,2016-01-01,1,1.877962
4,광주,2016-01-01,1,0.755054
...,...,...,...,...
12439,전남,2016-12-31,2,1.496123
12440,전북,2016-12-31,2,1.221665
12441,제주,2016-12-31,2,0.224163
12442,충남,2016-12-31,2,1.014732


In [248]:
verify['frequency'] = hap['Label']
verify

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,1.164587
1,2016-01-01,경기,1,4.519052
2,2016-01-01,경남,1,1.982569
3,2016-01-01,경북,1,1.877962
4,2016-01-01,광주,1,0.755054
...,...,...,...,...
12439,2016-12-31,전남,2,1.496123
12440,2016-12-31,전북,2,1.221665
12441,2016-12-31,제주,2,0.224163
12442,2016-12-31,충남,2,1.014732


In [249]:
verify[verify['frequency']<0] = 0

In [250]:
verify.to_csv('Z:/공모전/날씨빅데이터콘테스트(공유)/검증데이터셋제출/220023_es_0803_5.csv', encoding='cp949', index=False)

In [131]:
verify

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,1.305771
1,2016-01-01,경기,1,4.618941
2,2016-01-01,경남,1,2.069428
3,2016-01-01,경북,1,1.953093
4,2016-01-01,광주,1,0.923844
...,...,...,...,...
12439,2016-12-31,전남,2,1.521574
12440,2016-12-31,전북,2,1.245056
12441,2016-12-31,제주,2,0.264006
12442,2016-12-31,충남,2,0.900732
